In [ ]:
# Change to your own
PATH_NAME = '/content/drive/MyDrive/diet_rec_stuff/'

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pandas as pd
import os
from nltk.stem import WordNetLemmatizer
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
import re
from collections import Counter

In [ ]:
# Load word2vec model
model = KeyedVectors.load(PATH_NAME + "model_added_sentences.kv", mmap='r')

In [ ]:
# Getting a sample of what an embedding looks like
model["wheat"]

array([-1.76410004e-02,  2.55663097e-01, -3.16390663e-01,  3.11150372e-01,
        2.95398265e-01, -1.24751069e-01,  1.32025528e+00, -1.12339616e+00,
        3.28217685e-01, -8.14881176e-02,  4.24378067e-01, -3.32381099e-01,
       -6.86728358e-02,  3.30607414e-01, -8.94472972e-02,  3.35673541e-01,
       -2.18889713e-01, -3.32532853e-01, -1.02531321e-01, -5.97699046e-01,
       -1.04507208e-01,  7.09379792e-01,  2.70595402e-01, -4.14448798e-01,
        4.11527455e-01,  5.96357167e-01, -8.45980123e-02,  4.17825997e-01,
       -1.26356095e-01,  8.38835537e-02, -2.29937378e-02, -8.34348321e-01,
        3.71698588e-01,  5.05485594e-01, -8.44634473e-01,  6.00706935e-01,
        4.35924947e-01, -8.84729326e-02,  5.59986234e-01,  9.87276971e-01,
       -1.53232083e-01, -2.79031366e-01,  1.51084483e-01, -7.74627149e-01,
       -9.98749509e-02, -1.12253927e-01,  3.95807236e-01,  3.90611328e-02,
       -6.37600899e-01, -9.66136098e-01, -9.07164142e-02, -4.74767148e-01,
       -5.05510271e-01, -

In [ ]:
# Load WWEIA food categories csv. Contains the food category codes and associated verbal food categories.
# The "same_category" column lists which categories are considered synonymous.
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_food_categories

,wweia_food_category_code,wweia_food_category_description,larger_category,same_category
0,1002,"Milk, whole",Dairy,Milk
1,1004,"Milk, reduced fat",Dairy,Milk
2,1006,"Milk, lowfat",Dairy,Milk
3,1008,"Milk, nonfat",Dairy,Milk
4,1202,"Flavored milk, whole",Dairy,Milk
...,...,...,...,...
150,9404,"Formula, prepared from powder",Other,NaN
151,9406,"Formula, prepared from concentrate",Other,NaN
152,9602,Human milk,Other,NaN
153,9802,Protein and nutritional powders,Other,Protein and nutritional powders


In [ ]:
# Load the vocabulary of WWEIA food descriptions

food_words = pd.read_csv(PATH_NAME + 'food_words.csv')
food_words = food_words.iloc[:,1]
food_words = food_words.to_list()

In [ ]:
# Load wweia_data, which contains the FNDDS foods, their nutritional information, and their food category

wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')
wweia_data

,fdc_id,description,wweia_category_code,protein_kcal,total_lipid_kcal,carbohydrate_kcal,energy_kcal,dietary_fiber_g,calcium_mg,iron_mg,magnesium_mg,phosphorus_mg,potassium_mg,sodium_mg,zinc_mg,copper_mcg,selenium_mcg,Vitamin_A_mg_RAE,Vitamin_E_mg_AT,Vitamin_C_mg,thiamin_mg,riboflavin_mg,niacin_mg,Vitamin_B6_mg,Vitamin_B12_mcg,choline_mg,folate_mcg_DFE
0,781082,"Milk, human",9602,3.811,38.106,26.182,70,0.0,32,0.03,3,14,51,17,0.17,0.052,1.8,61,0.08,5.0,0.014,0.036,0.177,0.011,0.05,16.0,5
1,781083,"Milk, NFS",1004,12.099,17.748,18.430,51,0.0,119,0.03,11,91,141,45,0.42,0.014,3.1,53,0.04,0.1,0.039,0.179,0.091,0.037,0.49,15.8,5
2,781084,"Milk, whole",1002,11.655,28.275,18.240,61,0.0,113,0.03,10,84,132,43,0.37,0.025,3.7,46,0.07,0.0,0.046,0.169,0.089,0.036,0.45,14.3,5
3,781085,"Milk, low sodium, whole",1002,11.470,30.102,16.948,61,0.0,101,0.05,5,86,253,3,0.38,0.010,2.0,29,0.08,0.9,0.020,0.105,0.043,0.034,0.36,16.0,5
4,781086,"Milk, calcium fortified, whole",1002,11.655,28.275,18.240,61,0.0,113,0.03,10,84,132,43,0.37,0.025,3.7,46,0.07,0.0,0.046,0.169,0.089,0.036,0.45,14.3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,789767,Breading or batter as ingredient in food,9999,23.384,12.093,150.632,201,1.6,47,2.57,14,79,71,246,0.50,0.093,18.6,8,0.09,0.0,0.428,0.274,3.153,0.039,0.08,21.2,143
8686,789768,Wheat bread as ingredient in sandwiches,9999,42.365,34.452,172.102,263,5.4,161,3.12,56,166,190,464,1.36,0.191,27.6,0,0.35,0.1,0.398,0.214,4.886,0.159,0.00,21.3,82
8687,789769,Sauce as ingredient in hamburgers,9999,4.773,198.795,65.322,271,0.6,18,0.43,13,33,190,845,0.21,0.064,4.5,21,1.90,2.5,0.027,0.112,0.917,0.104,0.04,20.0,8
8688,789770,Industrial oil as ingredient in food,9999,0.000,870.000,0.000,892,0.0,0,0.04,0,0,0,0,0.01,0.000,0.0,0,10.50,0.0,0.000,0.000,0.000,0.000,0.00,0.2,0


In [ ]:
# Load embeddings
wweia_embeddings = np.loadtxt(PATH_NAME + 'wweia_embeddings_added_sen.csv', delimiter = ",")
wweia_embeddings.shape

(8690, 300)

In [ ]:
def get_most_pop_categories(curr_log, n, col_name):

  cats = curr_log.loc[:, col_name]

  cats_counter = Counter(cats)
  counter_results = cats_counter.most_common(n)
  
  counter_list = [elem[0] for elem in counter_results]
  
  return counter_list

In [ ]:
# initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Only keep words in phrase that are in food_words

def reduce_with_food_words(comma_phrase):
  comma_phrase_reduced = [lemmatizer.lemmatize(word.lower()) for word in comma_phrase.split() if (lemmatizer.lemmatize(word.lower()) in food_words or word.lower() in food_words)]
  return comma_phrase_reduced

In [1]:
# Format of food names seems to be (Optional) Company, Food, Details about food
# run function to guess if First or second comma phrase has Food

def process_food_log(curr_log, wweia_synonym_cats):

  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    descrip = curr_log.loc[i, 'Food Name']
    descrip_split = descrip.split(", ")

    # Reduce food log description to words in food_words
    first_phrase_num_words = reduce_with_food_words(descrip_split[0])

    pre_embedding = []

    if len(descrip_split) > 1:

      second_phrase_num_words = reduce_with_food_words(descrip_split[1])
      
      descrip_split_0_words = descrip_split[0].split()
      descrip_split_1_words = descrip_split[1].split()

      begin_first = False if len(second_phrase_num_words)/len(descrip_split_1_words) > len(first_phrase_num_words)/len(descrip_split_0_words) else True
      
      if begin_first:
        last_phrase = descrip

      if begin_first == False:
        three_phrases = descrip.partition(", ")
        last_phrase = three_phrases[2]

    else:
      last_phrase = descrip

    last_phrase_comma = last_phrase.split(", ")
    for j in range(len(last_phrase_comma)):
      split_comma = last_phrase_comma[j].split(" ")
      for elem in split_comma:
        elem = elem.lower()
        pre_embedding.append(elem)

    word_embed = np.zeros(shape = (1, len(model["sushi"])))

    if len(pre_embedding) > 0:
      # Turned edited_descrip into embedding vector by averaging the words
      word_embed = np.zeros(shape = (1, len(model["sushi"])))
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words

    # Compare to the other vectors
    similarities = cosine_similarity(word_embed, wweia_embeddings) # (1, 7918) for each wweia food

    # Finding WWEIA foods with highest similarity with each
    # food log food. Row has sorted list of WWEIA food IDs 
    # for each food log food.
    to_keep = np.sort(similarities, axis=1)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)
    
    sym_rank = 1000000

    # record RR here
    for index in range(indices.shape[1]):
      true_cat = curr_log.loc[i, 'wweia_food_category_code']
      if math.isnan(true_cat): continue
      if wweia_data.loc[indices[0,index], 'wweia_category_code'] == true_cat:
        rank = index
        rr = 1 / (rank+1)
        if sym_rank > rank:
          sym_rank = rank
          sym_rr = 1 / (rank+1)

        break
      else:
        if wweia_data.loc[indices[0,index], 'wweia_category_code'] in wweia_synonym_cats[true_cat]:
          if sym_rank > index:
            sym_rank = index
            sym_rr = 1 / (index+1)
    
    most_sim_food_index = indices[0,0]
    
    most_sim_food_row = wweia_data.iloc[most_sim_food_index,:]
    highest_cat_num = most_sim_food_row['wweia_category_code']
    
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['wweia_food_category_code'] == highest_cat_num, 'wweia_food_category_description']

    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])
    curr_log.loc[i, 'most_sim_food'] = most_sim_food_row['description']
    curr_log.loc[i, 'reciprocal_rank'] = rr
    curr_log.loc[i, 'sym_reciprocal_rank'] = sym_rr
    
  return curr_log

In [ ]:
def get_metrics(curr_log, wweia_synonym_cats):

  # Will hold foods already seen so foods aren't double-counted
  # when calculating accuracy
  seen_foods = set()
  total_valid_foods = 0
  num_corr = 0
  num_sym = 0

  for i in range(curr_log.shape[0]):
    
    true_cat = curr_log.loc[i, 'wweia_food_category_code']
    false_cat = curr_log.loc[i, 'predicted_categories_number']

    prev_set_size = len(seen_foods)
    seen_foods.add(curr_log.loc[i, 'Food Name'])

    if math.isnan(true_cat): continue # food does not have valid category, i.e. antacid

    if len(seen_foods) > prev_set_size: # not a repeated food

      if true_cat == false_cat:
        num_corr += 1
        num_sym += 1

      else:
        if false_cat in wweia_synonym_cats[true_cat]:
          num_sym += 1
        
      total_valid_foods += 1

  acc = num_corr / total_valid_foods
  sym_acc = num_sym / total_valid_foods

  return acc, sym_acc

In [ ]:
def get_food_prefs(food_log_categories, wweia_cat_nums_to_words):

  wweia_food_cats_list = wweia_food_categories['wweia_food_category_code'].tolist()

  # Make dict to hold the number of foods that belong to a
  # specific WWEIA category. Key is the broad food category.
  cats_dict = {}
  cats_dict["Protein"] = defaultdict(int)
  cats_dict["Vegetable"] = defaultdict(int)
  cats_dict["Grain"] = defaultdict(int)
  cats_dict["Fruit"] = defaultdict(int)
  cats_dict["Dairy"] = defaultdict(int)

  # Dict with lists containing cat codes for each broad food category
  category_mapping = { "Protein" : wweia_food_cats_list[14:34], "Vegetable" : wweia_food_cats_list[91:104],
  "Grain" : wweia_food_cats_list[35:81], "Fruit" : wweia_food_cats_list[82:90], "Dairy" : wweia_food_cats_list[0:13] }

  broad_categories_list = ["Protein", "Vegetable", "Grain", "Fruit", "Dairy"]

  for category_code in food_log_categories:

    if math.isnan(category_code): continue # food does not have valid category, i.e. antacid

    # Loop through each category code and add 1 to its dict entry
    # if it's the right one
    for broad_cat in broad_categories_list:
      if category_code in category_mapping[broad_cat]:
        cats_dict[broad_cat][category_code] += 1
        continue

  max_code_dict = {}
  
  for broad_cat in broad_categories_list:

    if len(list(cats_dict[broad_cat].keys())) > 0:
      max_code_dict[broad_cat] = max(cats_dict[broad_cat], key=cats_dict[broad_cat].get)

    else:
      max_code_dict[broad_cat] = "None of this category was eaten"

  return max_code_dict

In [ ]:
def make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats):
  
  acc = 0
  sym_acc = 0

  for key in true_food_prefs:
    if true_food_prefs[key] == pred_food_prefs[key]:
      acc += 1
      sym_acc += 1
    else:
      if true_food_prefs[key] not in wweia_synonym_cats: continue
      elif pred_food_prefs[key] in wweia_synonym_cats[true_food_prefs[key]]:
        sym_acc += 1

  acc = acc / len(list(true_food_prefs.keys()))
  sym_acc = sym_acc / len(list(true_food_prefs.keys()))

  return acc, sym_acc

In [ ]:
def get_pref_metrics(curr_log, n):
  
  true_pop_cats = get_most_pop_categories(curr_log, n, 'wweia_food_category_code')
  pref_pop_cats = get_most_pop_categories(curr_log, n, 'predicted_categories_number')

  true_pop_cats_sym = set()
  pref_pop_cats_sym = set()

  for elem in true_pop_cats:
    for sym_elem in wweia_synonym_cats[elem]:
      true_pop_cats_sym.add(sym_elem)

  for elem in pref_pop_cats:
    for sym_elem in wweia_synonym_cats[elem]:
      pref_pop_cats_sym.add(sym_elem)

  cats_common = set(true_pop_cats).intersection(set(pref_pop_cats))
  percent_common = len(cats_common) / n

  cats_common_sym = set(true_pop_cats_sym).intersection(set(pref_pop_cats_sym))
  percent_common_sym = min(1, len(cats_common_sym) / n)

  return percent_common, percent_common_sym

In [ ]:
# Main method

avg_acc = 0.0
avg_sym_acc = 0.0
avg_pref_acc = 0.0
avg_pref_sym_acc = 0.0
avg_percent_common = 0.0
avg_percent_common_sym = 0.0
avg_mean_rr = 0.0
avg_sym_mean_rr = 0.0

food_log_names = []
accs = []
sym_accs = []
pref_accs = []
pref_sym_accs = []
percent_common_list = []
percent_common_sym_list = []
mean_rrs = []
sym_mean_rrs = []

path = PATH_NAME + "sample_food_logs"

for food_log_name in os.listdir(path):
  
  curr_log = pd.read_csv(os.path.join(path, food_log_name))

  with open(PATH_NAME + 'wweia_synonym_cats.pickle', 'rb') as handle:
    wweia_synonym_cats = pickle.load(handle)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  food_log_names.append(food_log_name)

  # # Write to file saved in Google Drive folder
  # file_name = PATH_NAME + 'lfp_method4/' + food_log_name
  # curr_log.to_csv(file_name)

  # Compare to true food categories
  acc, sym_acc = get_metrics(curr_log, wweia_synonym_cats)
  avg_acc += acc
  avg_sym_acc += sym_acc
  accs.append(acc)
  sym_accs.append(sym_acc)
  print("Accuracy", acc, "Average Synonymous Accuracy:", sym_acc)

  # Count most popular categories and get food preferences
  with open(PATH_NAME + 'wweia_cat_nums_to_words.pickle', 'rb') as handle:
    wweia_cat_nums_to_words = pickle.load(handle)

  percent_common, percent_common_sym = get_pref_metrics(curr_log, 10)
  print("Percent Top N Categories Identified:", percent_common, "Percent Top N Synonymous Categories Identified:", percent_common_sym)
  avg_percent_common += percent_common
  avg_percent_common_sym += percent_common_sym
  percent_common_list.append(percent_common)
  percent_common_sym_list.append(percent_common_sym)

  true_food_prefs = get_food_prefs(curr_log.loc[:,'wweia_food_category_code'].to_list(), wweia_cat_nums_to_words)
  pred_food_prefs = get_food_prefs(curr_log.loc[:,'predicted_categories_number'].to_list(), wweia_cat_nums_to_words)

  # print(true_food_prefs)
  # print(pred_food_prefs)

  pref_acc, pref_sym_acc = make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats)
  avg_pref_acc += pref_acc
  avg_pref_sym_acc += pref_sym_acc
  pref_accs.append(pref_acc)
  pref_sym_accs.append(pref_sym_acc)
  print("Preference Accuracy:", pref_acc, "Preference Synonymous Accuracy:", pref_sym_acc)

  mean_rr = np.mean(curr_log.loc[:,'reciprocal_rank'])
  sym_mean_rr = np.mean(curr_log.loc[:,'sym_reciprocal_rank'])
  avg_mean_rr += mean_rr
  avg_sym_mean_rr += sym_mean_rr
  mean_rrs.append(mean_rr)
  sym_mean_rrs.append(sym_mean_rr)
  print("Mean Reciprocal Rank:", mean_rr, "Synonymous Mean Reciprocal Rank:", sym_mean_rr)

# # Averaged statistics for all food logs
# avg_acc /= len(os.listdir(path))
# avg_sym_acc /= len(os.listdir(path))
# avg_pref_acc /= len(os.listdir(path))
# avg_pref_sym_acc /= len(os.listdir(path))
# avg_percent_common /= len(os.listdir(path))
# avg_percent_common_sym /= len(os.listdir(path))
# avg_mean_rr /= len(os.listdir(path))
# avg_sym_mean_rr /= len(os.listdir(path))
# print("Average Accuracy:", avg_acc, "Average Synonymous Accuracy:", avg_sym_acc)
# print("Average Mean Reciprocal Rank:", avg_mean_rr, "Average Synonymous Mean Reciprocal Rank:", avg_sym_mean_rr)
# print("Average Preference Accuracy:", avg_pref_acc, "Average Synonymous Preference Accuracy:", avg_pref_sym_acc)
# print("Average Percent Top N Categories Identified:", avg_percent_common, "Average Percent Top N Synonymous Categories Identified:", avg_percent_common_sym)


Accuracy 0.5555555555555556 Average Synonymous Accuracy: 0.5555555555555556
Percent Top N Categories Identified: 0.5 Percent Top N Synonymous Categories Identified: 0.8
Preference Accuracy: 0.6 Preference Synonymous Accuracy: 0.6
Mean Reciprocal Rank: 0.634090909090909 Synonymous Mean Reciprocal Rank: 0.634090909090909


In [ ]:
# Make table to show accuracies
df = pd.DataFrame(list(zip(food_log_names, accs, sym_accs, pref_accs, pref_sym_accs, percent_common_list, percent_common_sym_list, mean_rrs, sym_mean_rrs)),
               columns =['Food Log', 'Accuracies', "Synomynous Accuracies", "Preference Accuracies", "Synomynous Preference Accuracies", "Percent of Top N Categories Achieved", "Percent of Top N Categories Achieved Sym", "Mean RR", "Sym Mean RR"])
df

,Food Log,Accuracies,Synomynous Accuracies,Preference Accuracies,Synomynous Preference Accuracies,Percent of Top N Categories Achieved,Percent of Top N Categories Achieved Sym,Mean RR,Sym Mean RR
0,sample_food_log.csv,0.555556,0.555556,0.6,0.6,0.5,0.8,0.634091,0.634091
